# Projeto 1 - Ciência dos Dados

Nome: Camila Soares Magni

Nome: Lincoln Rodrigo Pereira Melo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [35]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [36]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\camil\OneDrive\Área de Trabalho\INSPER\2° sem\C dados\Projeto1- Cdados\Proj1Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [37]:
filename = 'airfryer.xlsx'

In [38]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,o twitter vai me fazer comprar uma airfryer,1
1,"quero assar pão de queijo, mas o forno daqui d...",1
2,"bom, aparentemente deu ruim e saiu toda a tint...",1
3,de airfryer a panelas: polishop celebra 22 ano...,0
4,@danielsmarconn mano eu tô louca nessa airfrye...,1


In [39]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,"o bom da airfryer fica tudo pronto rápido, não...",1
1,um app pra calcular se é mais barato fazer a c...,0
2,@raytotsukishiro aumentou o preço de tudo. aqu...,0
3,meti um pão de queijo na airfryer 😌,1
4,"@alcysio @safbf pô alcysio, me adaptei total p...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos a fritadeira sem óleo airfryer como nosso produto. Para a classificação dos tweets, consideramos todas as mensagens que expressam alguma opinião sobre o produto, sejam elogios ou críticas, como relevantes, assim como as que falam sobre um desejo de adquirir uma airfryer. O resto foi classificado como irrelevante, incluindo receitas e mensagens sobre outros assuntos que não fazem sentido para a análise do produto. 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [40]:
import pandas as pd
import numpy as np
from IPython.display import display
import re 
import nltk

# Função de limpeza dos textos
def cleanup(text):
    
    punctuation = '[""''!-.:?;“”~1234567890]'                                      # sinais que queremos retirar do texto
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)                              # retira os sinais
    text_subbed = re.sub(r"http\S+", "", text_subbed)                    # retira os URL
    text_subbed = re.sub('@[^\s]+','',text_subbed,flags=re.MULTILINE)    # retira os nomes de usuário
    text_subbed = re.sub(r"kk\S+", "kk", text_subbed)                    # transformar as strings que começam com kk em 'kk'
    text_subbed = re.sub(r"compr\S+", "compr", text_subbed) 
    text_subbed = re.sub(r"limp\S+", "limp", text_subbed) 
    text_subbed = re.sub(r"aa\S+", "", text_subbed)
    text_subbed = re.sub(r"airfryer\S+", "airfryer", text_subbed)
    text_subbed = re.sub(r"queij\S+", "queij", text_subbed)
    text_subbed = re.sub(r"batat\S+", "batata", text_subbed) 
    aux = nltk.TweetTokenizer()
    text_subbed = aux.tokenize(text_subbed)                              # separa os emojis
    string_final = ' '.join(text_subbed)                                 # string limpa
    return string_final

In [41]:
# Função para tirar as 'Stopwords' em português:
# nltk.download('stopwords') - INSTALAR, caso vc não tenha instalado
def retirar_stopwords(lista):
    lista_limpa = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for palavra in lista:
        if palavra not in stopwords:
            lista_limpa.append(palavra)
    return lista_limpa

In [43]:
lista = []

for linha in train['Treinamento']:
    limpo = cleanup(linha.lower())             # Limpa a planilha Treinamento e tira as letras maiúsculas 
    a = limpo.split()
    b = retirar_stopwords(a)
    limpo = ' '.join(b)  
    lista.append(limpo)
    
serie_train = pd.Series(lista)                 # Transforma a coluna da planilha Treinamento em Series
serie_train.head()

0                     twitter vai fazer compr airfryer
1    quero assar pão queij forno daqui casa bosta p...
2    bom aparentemente deu ruim saiu toda tinta / t...
3    airfryer panelas polishop celebra anos descont...
4              mano tô louca nessa airfryer serase boa
dtype: object

In [45]:
train['Treinamento'] = serie_train        # substitui os tweets originais da planilha, pelos tweets limpos
train.head()

,Treinamento,Classificação
0,twitter vai fazer compr airfryer,1
1,quero assar pão queij forno daqui casa bosta p...,1
2,bom aparentemente deu ruim saiu toda tinta / t...,1
3,airfryer panelas polishop celebra anos descont...,0
4,mano tô louca nessa airfryer serase boa,1


In [46]:
filtro = train.Classificação==1          # filtra os tweets classificados como relevantes
relevantes = train.loc[filtro, :]        # dataframe com tweets relevantes

filtro2 = train.Classificação==0        # filtra os tweets classificados como irrelevantes
irrelevantes = train.loc[filtro2, :]    # dataframe com tweets irrelevantes

In [47]:
# Transforma o dataframe relevantes em um único texto
string_relevante = ''
for linha in relevantes.Treinamento:
    string_relevante+= ' '+ linha

In [48]:
# Transforma o dataframe irrelevantes em um único texto
string_irrelevante = ''
for linha in irrelevantes.Treinamento:
    string_irrelevante+= ' '+ linha

In [49]:
todas_relevantes = string_relevante.split()        #Lista com todas as palavras que aparece na string relevante
todas_irrelevantes = string_irrelevante.split()    #Lista com todas as palavras que aparece na string irrelevante

serie_relevante = pd.Series(todas_relevantes)      #Transforma a lista de strings relevantes em um series
serie_irrelevante = pd.Series(todas_irrelevantes)  #Transforma a lista de strings irrelevantes em um series

In [50]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa                                        # percentual de cada palavra da series relevante

airfryer        0.094178
batata          0.015982
compr           0.014840
pra             0.014840
q               0.012557
                  ...   
dourada         0.000571
experiências    0.000571
eletrônicos     0.000571
teflon          0.000571
cartao          0.000571
Length: 872, dtype: float64

In [51]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_irrelevante_relativa.head(50)                                        # percentual de cada palavra da series irrelevante

airfryer      0.099794
pra           0.009635
q             0.008947
casa          0.006882
kk            0.006194
aqui          0.006194
óleo          0.006194
compr         0.006194
usar          0.006194
hoje          0.005506
tô            0.005506
fazer         0.004818
pode          0.004818
mundo         0.004818
gás           0.004129
vida          0.004129
n             0.004129
melhor        0.004129
deus          0.004129
alho          0.004129
ter           0.004129
vai           0.004129
ser           0.004129
tava          0.004129
dentro        0.004129
mãe           0.004129
tudo          0.003441
luz           0.003441
pão           0.003441
fica          0.003441
fogão         0.003441
energia       0.003441
ventilador    0.003441
ficou         0.003441
coisas        0.002753
frango        0.002753
assim         0.002753
tipo          0.002753
✨             0.002753
vento         0.002753
seita         0.002753
lavar         0.002753
usando        0.002753
fazendo    

In [52]:
todas = string_irrelevante + string_relevante          # juntando todas as palavras que aparecem na base de dados Treinamento
todas_as_palavras = todas.split()                      # lista com todas as palavras
serie_total = pd.Series(todas_as_palavras) 
serie_total
# tranforma a lista em Series
# tabela_total_relativa = serie_total.value_counts(True)
# tabela_total_relativa                                   # percentual de cada palavra em relação a todas as palavras

0       airfryer
1        panelas
2       polishop
3        celebra
4           anos
          ...   
3200       ranço
3201      batata
3202      palito
3203       feita
3204    airfryer
Length: 3205, dtype: object

In [53]:
P_R = len(todas_relevantes)/len(todas_as_palavras)        # probabilidade de uma palavra ser relevante
P_I = len(todas_irrelevantes)/len(todas_as_palavras)      # probabilidade de uma palavra ser irrelevante
P_I + P_R                                                 # soma das probabilidade para verificar se está correto

1.0

In [54]:
freq_rel_absoluta = serie_relevante.value_counts()           # quantidade de cada palavra na serie relevante
freq_irrel_absoluta = serie_irrelevante.value_counts()       # quantidade de cada palavra na serie irrelevante
freq_tot_absoluta = serie_total.value_counts()               # quantidade de cada palavra na serie total

# Função Classificadora Naive-Bayes

In [55]:
def classificador(dataframe, series):                         #recebe o dataframe e a serie de tweets que será classificada
    
    classificados = []
    for frase in series:
        lista_palavras = frase.split()
        lista_palavras = retirar_stopwords(lista_palavras)    #implementa função que tira as stopwords
        k,i = 0,0                                             #contador de palavras NÃO presentes em relevantes (k) e em irrevelantes (i)
        for palavra in lista_palavras:
            if palavra not in todas_relevantes: 
                k += 1
            if palavra not in todas_irrelevantes:
                i += 1

        prob_frase_dado_relevante = 1
        prob_frase_dado_irrelevante = 1

        if k > 0:                                   #se tiver palavras no series que não estao na nossa base de dados relevantes
            for palavra in lista_palavras:
                if palavra in freq_rel_absoluta:
                    prob = (freq_rel_absoluta[palavra]+ 1 ) / (freq_rel_absoluta.sum()+ len(freq_tot_absoluta)) # SUAVIZAÇÃO DE LAPLACE"
                else: 
                    prob = (0 + 1 ) / (freq_rel_absoluta.sum()+ len(freq_tot_absoluta))
                prob_frase_dado_relevante *= prob           #multiplica a probabilidade de cada palavra dado frase
        else:
            for palavra in lista_palavras:
                prob = freq_rel_absoluta[palavra] / freq_rel_absoluta.sum()
                prob_frase_dado_relevante *= prob           #multiplica a probabilidade de cada palavra dado frase

        if i > 0:
            for palavra in lista_palavras:
                if palavra in freq_irrel_absoluta:
                    prob = ( freq_irrel_absoluta[palavra]+ 1 ) / (freq_irrel_absoluta.sum() +len(freq_tot_absoluta))
                else:
                    prob = ( 0 + 1 ) / (freq_irrel_absoluta.sum() +len(freq_tot_absoluta))
                prob_frase_dado_irrelevante *= prob         #multiplica a probabilidade de cada palavra dado frase
        else:
            for palavra in lista_palavras:
                prob = ( freq_irrel_absoluta[palavra] ) / (freq_irrel_absoluta.sum())
                prob_frase_dado_irrelevante *= prob         #multiplica a probabilidade de cada palavra dado frase
                
        probRdadoFrase = prob_frase_dado_relevante* P_R
        probIdadoFrase = prob_frase_dado_irrelevante* P_I
        
        if probRdadoFrase > probIdadoFrase:
            classificados.append(1)                         #classifica como relevante
        else:  
            classificados.append(0)                         #classifica como irrelevante
            
    dataframe['Classificado'] = pd.Series(classificados)    #adiciona uma nova coluna ao dataframe com a classificação
    return dataframe
                

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [56]:
test

,Teste,Classificação
0,"o bom da airfryer fica tudo pronto rápido, não...",1
1,um app pra calcular se é mais barato fazer a c...,0
2,@raytotsukishiro aumentou o preço de tudo. aqu...,0
3,meti um pão de queijo na airfryer 😌,1
4,"@alcysio @safbf pô alcysio, me adaptei total p...",1
...,...,...
195,tutorial bolinha de queijo✨\n\n200g de queijo ...,1
196,pessoal daqui de casa inventou de querer fazer...,1
197,"eu nao sei voces, mas eu ganhei uma airfryer g...",1
198,@ibebelly amanhã eu vou fazer um pão de alho d...,1


In [57]:
lista = []

for linha in test['Teste']:
    limpo = cleanup(linha.lower())             # Limpa a planilha Teste e tira as letras maiúsculas
    a = limpo.split()
    b = retirar_stopwords(a)
    limpo = ' '.join(b)  
    lista.append(limpo)

test['Teste'] = pd.Series(lista)             # Substitui os tweets originais da planilha, pelos tweets limpos
test

,Teste,Classificação
0,bom airfryer fica tudo pronto rápido quero out...,1
1,app pra calcular barato fazer comida airfryer ...,0
2,aumentou preço tudo aqui gente compr airfryer ...,0
3,meti pão queij airfryer 😌,1
4,pô alcysio adaptei total pra usar mínimo gás a...,1
...,...,...
195,tutorial bolinha queij g queij muçarela ralado...,1
196,pessoal daqui casa inventou querer fazer tudo ...,1
197,nao sei voces ganhei airfryer gigante,1
198,amanhã vou fazer pão alho airfryer mandar foto...,1


# Implementando a função...

In [58]:
df = test               # dataframe que será argumento da função classificadora
series= test['Teste']   # series que será argumento da função classificadora

In [59]:
# Implementa função
df_classificado = classificador(df, series)
df_classificado

,Teste,Classificação,Classificado
0,bom airfryer fica tudo pronto rápido quero out...,1,1
1,app pra calcular barato fazer comida airfryer ...,0,1
2,aumentou preço tudo aqui gente compr airfryer ...,0,0
3,meti pão queij airfryer 😌,1,1
4,pô alcysio adaptei total pra usar mínimo gás a...,1,1
...,...,...,...
195,tutorial bolinha queij g queij muçarela ralado...,1,0
196,pessoal daqui casa inventou querer fazer tudo ...,1,1
197,nao sei voces ganhei airfryer gigante,1,1
198,amanhã vou fazer pão alho airfryer mandar foto...,1,1


In [60]:
filtro_1 = df_classificado.Classificação == 1
nova = df_classificado.loc[filtro_1]
nova    #planilha que classificamos como relevante

,Teste,Classificação,Classificado
0,bom airfryer fica tudo pronto rápido quero out...,1,1
3,meti pão queij airfryer 😌,1,1
4,pô alcysio adaptei total pra usar mínimo gás a...,1,1
5,vamos tentar fritar batata airfryer,1,1
6,dia vc vai mandar airfryer aqui casa,1,1
...,...,...,...
195,tutorial bolinha queij g queij muçarela ralado...,1,0
196,pessoal daqui casa inventou querer fazer tudo ...,1,1
197,nao sei voces ganhei airfryer gigante,1,1
198,amanhã vou fazer pão alho airfryer mandar foto...,1,1


In [61]:
filtro1_new = df_classificado.Classificado == 1
new = df_classificado.loc[filtro1_new]
new    #planilha que foi classificada pela função como relevante

,Teste,Classificação,Classificado
0,bom airfryer fica tudo pronto rápido quero out...,1,1
1,app pra calcular barato fazer comida airfryer ...,0,1
3,meti pão queij airfryer 😌,1,1
4,pô alcysio adaptei total pra usar mínimo gás a...,1,1
5,vamos tentar fritar batata airfryer,1,1
...,...,...,...
191,vou fazer base pão queij airfryer,1,1
196,pessoal daqui casa inventou querer fazer tudo ...,1,1
197,nao sei voces ganhei airfryer gigante,1,1
198,amanhã vou fazer pão alho airfryer mandar foto...,1,1


## Extraindo as contagens para verificação de performance

In [62]:
tabela = pd.crosstab(df_classificado.Classificado, df_classificado.Classificação, normalize= True)  # tabela de porcentagens
tabela

Classificação,0,1
Classificado,,
0,0.28,0.095
1,0.17,0.455


Porcentagem de verdadeiros positivos (mensagens relevantes e que são
classificadas como relevantes)

In [63]:
verdadeiros_positivos = tabela.iloc[1,1]*100
verdadeiros_positivos

45.5

Porcentagem de falsos positivos (mensagens irrelevantes e que são classificadas
como relevantes)

In [64]:
falsos_positivos = tabela.iloc[1,0]*100
falsos_positivos

17.0

Porcentagem de verdadeiros negativos (mensagens irrelevantes e que são
classificadas como irrelevantes)

In [65]:
verdadeiros_negativos = tabela.iloc[0,0]*100
verdadeiros_negativos

28.000000000000004

Porcentagem de falsos negativos (mensagens relevantes e que são classificadas
como irrelevantes)

In [66]:
falsos_negativos = tabela.iloc[0,1]*100
falsos_negativos

9.5

Acurácia (mensagens corretamente classificadas, independente da categoria)

In [67]:
eficacia = verdadeiros_positivos + verdadeiros_negativos
eficacia

73.5

In [68]:
filtro1_new = df_classificado.Classificado == 1
new = df_classificado.loc[filtro1_new]
filtro2 = new.Classificação == 0 
df_falso_positivo = new.loc[filtro2]
df_falso_positivo

,Teste,Classificação,Classificado
1,app pra calcular barato fazer comida airfryer ...,0,1
8,agora sei uso airfryer fogão faca espada,0,1
9,melhor coisa aprendi twitter colocar papel alu...,0,1
17,chu morte 😭,0,1
20,rt menos airfryer lego,0,1
44,airfryer amigão outra coisa aí dá,0,1
45,entendo irmão,0,1
54,utero queij suíço utero airfryer utero liquidi...,0,1
59,airfryer milagrosa né coloquei sementinha lá a...,0,1
62,sentindo ainda adulto finalmente compr airfrye...,0,1


In [69]:
filtro_1 = df_classificado.Classificação == 1
nova = df_classificado.loc[filtro_1]
nova    #planilha que classificamos como relevante
filtro_2 = nova.Classificado == 0
df_falso_negativo = nova.loc[filtro_2]
df_falso_negativo

,Teste,Classificação,Classificado
16,quanto amam airfryer pq tô levando junto viagem,1,0
36,bonitona aqui agora airfryer oster favor so di...,1,0
42,ganhei airfryer empresa dei pra mãe tá toda bo...,1,0
49,cada dia radicalizando seita airfryer meti jer...,1,0
52,casa carro ações previdência privada esqueça c...,1,0
87,vadias dizem fans airfryer vão embora antes pr...,1,0
90,ranço fritar coisas óleo pq fica tudo virado ó...,1,0
95,café único continua mesma faixa preço olha óle...,1,0
110,esquentar pizza airfryer outro mundo,1,0
117,airfryer rápido prático amo,1,0


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**